In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import modules as mod

### Global parameters

In [2]:
# Number of market participants 
N = 1600 

# Number of fundamentalists and chartists
N_FUND = 800
N_CHART = 800

# Initial price of the asset traded
P_0 = 100

# Global fundamental price 
P_F = 120

# Variation around the global fundamental price 
THETA = 30

# Maximum extension for chartists' timesteps window 
T_MAX = 15

# Sensitivity of forecasts for fundamentalists
PHI = 2.0

# Sensitivity of forecasts for chartists 
KAPPA = 2.0

# level of information dissipation among the agents
ALPHA = 0.95

# Weight of the market imbalance
DELTA = 0.05

# Intervals of variation for the stochastic noise term 
SIGMA = 30

# Sensitivity threshold for choosing the status
TAO = 20

# Asset quantity for each trader
Q = 50

# Information Threshhold
Ith = 1

# Initializing traders' wealth (W), money (M), and asset quantity(Q)
M = 35000  
Q = 50 
W = [M + Q * P_0 for _ in range(N)] 

In [3]:
# Function to calculate expected price for one fundamentalist
def ep_fundamentalist(p_t, P_F):
    """
    Calculate the expected price for a fundamentalist trader

    Parameters:
    p_t (float): The current market price of the asset
    p_f (float): The perceived fundamental price of the asset for the trader

    Returns:
    float: The expected price for the asset for a fundamentalist trader
    """
    noise = np.random.uniform(-SIGMA, SIGMA)
    # Was not too sure about using an uniform or normal distribution here
    p_f = np.random.uniform(P_F - THETA, P_F + THETA)
    return p_t + PHI * (p_f - p_t) + noise

# Function to calculate expected price for one chartist
def ep_chartist(p_t, past_prices):
    """
    Calculate the expected price for a chartist trader

    Parameters:
    p_t (float): The current market price of the asset
    past_prices (list of float): A list of past market prices of the asset

    Returns:
    float: The expected price for the asset as calculated by a chartist trader
    """
    assert len(past_prices) == T_MAX, "past_prices must be a list of 15 values"
    chartist_T = np.random.randint(2, T_MAX)
    p_T = np.mean(past_prices[-chartist_T:]) 
    noise = np.random.uniform(-SIGMA, SIGMA)
    return p_t + KAPPA/(chartist_T) * (p_t - p_T) + noise

# Function that determines the choice one random trader makes
def rand_trader():
    choice = np.random.randint(0, 3)
    return choice
    

### 

In [4]:
# Function to calculate the order type for a non-random trader
def determine_order_type(expected_price, p_t, TAO, money, asset_quantity):
    """
    Determine the order type of a trader based on the expected price, current price,
    sensitivity threshold, available money, and asset quantity

    Parameters:
    expected_price (float): The expected price of the asset
    p_t (float): The current market price of the asset
    TAO (int): The threshold to decide whether to hold
    money (float): The amount of money the trader has
    asset_quantity (int): The quantity of the asset the trader holds

    Returns:
    str: The order type ('buy', 'sell', or 'hold')
    """
    if abs(expected_price - p_t) < TAO:
        return 'hold'
    elif expected_price > p_t and money > 0:
        return 'buy'
    elif expected_price < p_t and asset_quantity > 0:
        return 'sell'
    else:
        return 'hold'

def set_order_price(order_type, expected_price, current_price, money):
    """
    Set the order price for a trader based on their order type, expected price,
    current price, and the amount of money they have

    Parameters:
    order_type (str): The type of order ('buy', 'sell', or 'hold')
    expected_price (float): The expected price of the asset
    current_price (float): The current market price of the asset
    money (float): The amount of money the trader has
    
    Returns:
    float: The price set for the order
    """
    if order_type == 'buy':
        return np.random.uniform(0, min(money, expected_price))
    elif order_type == 'sell':
        return np.random.uniform(expected_price, current_price)
    else:
        return None

In [5]:
# Example usage
current_market_price = P_0
trader_money = M
trader_asset_quantity = Q

ep = ep_fundamentalist(current_market_price, P_F) 
order_type = determine_order_type(ep, P_0, TAO, trader_money, trader_asset_quantity)
order_price = set_order_price(order_type, ep, current_market_price, trader_money)

print(ep)
print(order_type)
print(order_price)

106.87195125823382
hold
None


In [6]:
# Initialize agent attributes at T = 0

# structure of agent data
# agents = np.zeros(N, dtype=[('type', 'U10'),# 0
#                             ('wealth', 'f8'),# 1
#                             ('money', 'f8'), # 2
#                             ('assets', 'f8'), # 3
#                             ('expected_price', 'f8'), # 4
#                             ('decision', 'U10'), # 5
#                             ('info', 'f8')]) # 6

agents_dict = {}
past_price = np.repeat(P_0, T_MAX)
for i in range(N_FUND):
    ep = ep_fundamentalist(current_market_price, P_F) 
    
    agents_dict[i] = ('fund',
                 W[i],
                 M,
                 Q,
                 ep,
                 determine_order_type(ep, P_0, TAO, trader_money, trader_asset_quantity),
                 np.random.uniform(0, Ith)
                )

# Start from where N_FUND stops for integers
for i in range(N_FUND, N_FUND+N_CHART):
    ep = ep_chartist(current_market_price, past_price)
    agents_dict[i] = ('chart',
                 W[i],
                 M,
                 Q,
                 ep,
                 determine_order_type(ep, P_0, TAO, trader_money, trader_asset_quantity),
                 np.random.uniform(0, Ith)
                )


In [56]:
class OrderBook:
    def __init__(self, delta, agents_dict):
        """
        Initialize the OrderBook with the given delta value.

        Parameters:
        - delta (float): Parameter for price adjustment.
        """
        self.bids = []  # List to store bid orders
        self.asks = []  # List to store ask orders
        self.delta = delta  # Parameter for price adjustment
        self.N_a = 0
        self.N_b = 0
        self.N_T = 0
        self.agents_dict = agents_dict
    
        
    def place_bid(self, trader_id, price):
        """
        Place a bid order in the order book.

        Parameters:
        - trader_id (int): ID of the trader placing the bid.
        - price (float): Bid price.
        """
        self.bids.append({'trader_id': trader_id, 'price': price})
        self.N_b += 1  # Increment the number of bidders
        
    def place_ask(self, trader_id, price):
        """
        Place an ask order in the order book.

        Parameters:
        - trader_id (int): ID of the trader placing the ask.
        - price (float): Ask price.
        """
        self.asks.append({'trader_id': trader_id, 'price': price})
        self.N_a += 1  # Increment the number of askers
        


    def match_orders(self):
        """
        Match bid and ask orders in the order book.

        Returns:
        - list: List of dictionaries representing matched transactions.
        """
        self.bids.sort(key=lambda x: x['price'], reverse=True)
        self.asks.sort(key=lambda x: x['price'])
        
        N_T = 0 
        while self.bids and self.asks:
            best_bid = self.bids[0]
            best_ask = self.asks[0]

            if best_bid['price'] >= best_ask['price']:
                # Remove matched orders
                p_L = self.asks[0]['price']
                self.agents_dict[self.asks[0]['trader_id']][3] -= 1 ## assets
                self.agents_dict[self.bids[0]['trader_id']][3] += 1 
                self.agents_dict[self.asks[0]['trader_id']][2] += p_L  
                self.agents_dict[self.bids[0]['trader_id']][2] -= p_L
                self.bids.pop(0)
                self.asks.pop(0)
                self.N_b -= 1
                self.N_a -= 1
                N_T += 1
                print(best_bid)
            
            else:
                break  # No more possible transactions

        return N_T, p_L

    def set_aggregate_price(self, prev_price, N_a, N_b):
        """
        Set the aggregate asset price at time t + 1 based on different cases.

        Parameters:
        - prev_price (float): Previous global asset price at time t.
        - N_a (int): Number of askers in the order book.
        - N_b (int): Number of bidders in the order book.

        Returns:
        - float: New global asset price at time t + 1.
        """
        print('PREV', prev_price)

        if N_a == 0 and N_b > 0:
            new_price = prev_price + self.delta * N_b
            print('1')
        elif N_a > 0 and N_b == 0:
            new_price = prev_price - self.delta * N_a
            print('2')
        elif 0 < N_a < N_b and self.bids[0]['price'] < self.asks[0]['price']:
            new_price = prev_price + self.delta * N_b
            print('3')
        elif 0 < N_a < N_b and self.bids[0]['price'] > self.asks[0]['price']:
            N_T,p_L = self.match_orders()
            new_price = p_L + self.delta * (N_b - N_T)
            print('4')
        elif 0 < N_b < N_a and self.bids[0]['price'] <= self.asks[0]['price']:
            new_price = prev_price - self.delta * N_a
            print('5')
        elif 0 < N_b < N_a and self.bids[0]['price'] > self.asks[0]['price']:
            N_T, p_L = self.match_orders()
            new_price = p_L - self.delta * (N_a - N_T)
            print('6')
        else:
            new_price = prev_price
            print('7')
        print('NEW', new_price)
        return new_price


### Simulation

In [57]:
# Initialize agent attributes at T = 0

# structure of agent data
# agents = np.zeros(N, dtype=[('type', 'U10'),# 0
#                             ('wealth', 'f8'),# 1
#                             ('money', 'f8'), # 2
#                             ('assets', 'f8'), # 3
#                             ('expected_price', 'f8'), # 4
#                             ('decision', 'U10'), # 5
#                             ('info', 'f8')]) # 6

agents_dict = {}
past_price = [P_0 for i in range(T_MAX)]
for i in range(N_FUND):
    ep = ep_fundamentalist(current_market_price, P_F) 
    
    agents_dict[i] = ['fund',
                 W[i],
                 M,
                 Q,
                 ep,
                 determine_order_type(ep, P_0, TAO, trader_money, trader_asset_quantity),
                 np.random.uniform(0, Ith)
                     ]

# Start from where N_FUND stops for integers
for i in range(N_FUND, N_FUND+N_CHART):
    ep = ep_chartist(current_market_price, past_price)
    agents_dict[i] = ['chart',
                 W[i],
                 M,
                 Q,
                 ep,
                 determine_order_type(ep, P_0, TAO, trader_money, trader_asset_quantity),
                 np.random.uniform(0, Ith)
                     ]

In [58]:
print(current_market_price)

100


In [59]:
# Simulation 1 cycle
ob = OrderBook(DELTA, agents_dict)
current_market_price = P_0
simulations = 100
# Check #5, decision in dict to get decision
for sim in range(simulations):
    for agent_id in agents_dict:
        decision = agents_dict[agent_id][5]
        if decision == "buy":
            ob.place_bid(agent_id, agents_dict[agent_id][4])
        elif decision == "sell":
            ob.place_ask(agent_id, agents_dict[agent_id][4])
    current_market_price = ob.set_aggregate_price(current_market_price,
                                                 ob.N_a,
                                                 ob.N_b)


PREV 100
{'trader_id': 62, 'price': 226.5475584383048}
{'trader_id': 90, 'price': 224.91054118237165}
{'trader_id': 478, 'price': 224.2585226868123}
{'trader_id': 610, 'price': 221.85706137196485}
{'trader_id': 621, 'price': 220.57623368519103}
{'trader_id': 700, 'price': 220.02779271076366}
{'trader_id': 15, 'price': 219.1682700965373}
{'trader_id': 247, 'price': 219.13536577764077}
{'trader_id': 513, 'price': 217.5924126050194}
{'trader_id': 143, 'price': 216.16496397993313}
{'trader_id': 248, 'price': 215.58660348567747}
{'trader_id': 192, 'price': 214.01064142732892}
{'trader_id': 60, 'price': 212.80679264103978}
{'trader_id': 40, 'price': 212.1777118686614}
{'trader_id': 258, 'price': 211.2772480043743}
{'trader_id': 479, 'price': 210.39597751847248}
{'trader_id': 370, 'price': 210.318381783142}
{'trader_id': 266, 'price': 209.66926211607225}
{'trader_id': 412, 'price': 209.45345786194036}
{'trader_id': 39, 'price': 208.60674384917075}
{'trader_id': 684, 'price': 208.4230229000023

{'trader_id': 415, 'price': 187.9238385622515}
{'trader_id': 346, 'price': 187.29236966617043}
{'trader_id': 524, 'price': 186.9736327638115}
{'trader_id': 774, 'price': 186.85365712376267}
{'trader_id': 593, 'price': 186.78144892779957}
{'trader_id': 455, 'price': 186.69772497186761}
{'trader_id': 190, 'price': 185.84046490841848}
{'trader_id': 456, 'price': 185.753200051513}
{'trader_id': 635, 'price': 185.48293356148878}
{'trader_id': 194, 'price': 185.46687322618618}
{'trader_id': 72, 'price': 184.90683440375886}
{'trader_id': 562, 'price': 184.86070970075198}
{'trader_id': 302, 'price': 184.7503870808638}
{'trader_id': 35, 'price': 184.55594195806708}
{'trader_id': 510, 'price': 184.04443503078423}
{'trader_id': 673, 'price': 183.9588991218164}
{'trader_id': 590, 'price': 183.85234751037552}
{'trader_id': 390, 'price': 183.7623774627735}
{'trader_id': 53, 'price': 183.67408347461063}
{'trader_id': 797, 'price': 183.59953740212768}
{'trader_id': 492, 'price': 183.5055967522526}
{'t

{'trader_id': 498, 'price': 201.05152477570908}
{'trader_id': 1, 'price': 200.57609005351622}
{'trader_id': 461, 'price': 200.43932554425044}
{'trader_id': 168, 'price': 199.5543481843553}
{'trader_id': 329, 'price': 199.22567991789094}
{'trader_id': 713, 'price': 199.18259554886782}
{'trader_id': 555, 'price': 198.86710152954285}
{'trader_id': 790, 'price': 198.86638021393867}
{'trader_id': 563, 'price': 198.67941961636996}
{'trader_id': 438, 'price': 198.6165575260751}
{'trader_id': 123, 'price': 198.4521484164236}
{'trader_id': 126, 'price': 198.26086608648114}
{'trader_id': 466, 'price': 197.84539913370503}
{'trader_id': 457, 'price': 197.6757451077879}
{'trader_id': 677, 'price': 197.3076823923814}
{'trader_id': 6, 'price': 197.20666247606698}
{'trader_id': 737, 'price': 197.20338856705922}
{'trader_id': 741, 'price': 196.69389649334653}
{'trader_id': 682, 'price': 196.0167745770529}
{'trader_id': 330, 'price': 195.945442795387}
{'trader_id': 181, 'price': 195.74509592874398}
{'tr

{'trader_id': 247, 'price': 219.13536577764077}
{'trader_id': 513, 'price': 217.5924126050194}
{'trader_id': 143, 'price': 216.16496397993313}
{'trader_id': 248, 'price': 215.58660348567747}
{'trader_id': 192, 'price': 214.01064142732892}
{'trader_id': 60, 'price': 212.80679264103978}
{'trader_id': 40, 'price': 212.1777118686614}
{'trader_id': 258, 'price': 211.2772480043743}
{'trader_id': 479, 'price': 210.39597751847248}
{'trader_id': 370, 'price': 210.318381783142}
{'trader_id': 266, 'price': 209.66926211607225}
{'trader_id': 412, 'price': 209.45345786194036}
{'trader_id': 39, 'price': 208.60674384917075}
{'trader_id': 684, 'price': 208.4230229000023}
{'trader_id': 37, 'price': 208.3911270953931}
{'trader_id': 476, 'price': 207.8234011976399}
{'trader_id': 762, 'price': 207.60145535897357}
{'trader_id': 326, 'price': 207.39730474671225}
{'trader_id': 158, 'price': 207.0875513784631}
{'trader_id': 70, 'price': 207.03248361684444}
{'trader_id': 251, 'price': 206.2592038891172}
{'trade

{'trader_id': 329, 'price': 199.22567991789094}
{'trader_id': 713, 'price': 199.18259554886782}
{'trader_id': 555, 'price': 198.86710152954285}
{'trader_id': 790, 'price': 198.86638021393867}
{'trader_id': 563, 'price': 198.67941961636996}
{'trader_id': 438, 'price': 198.6165575260751}
{'trader_id': 123, 'price': 198.4521484164236}
{'trader_id': 126, 'price': 198.26086608648114}
{'trader_id': 466, 'price': 197.84539913370503}
{'trader_id': 457, 'price': 197.6757451077879}
{'trader_id': 677, 'price': 197.3076823923814}
{'trader_id': 6, 'price': 197.20666247606698}
{'trader_id': 737, 'price': 197.20338856705922}
{'trader_id': 741, 'price': 196.69389649334653}
{'trader_id': 682, 'price': 196.0167745770529}
{'trader_id': 330, 'price': 195.945442795387}
{'trader_id': 181, 'price': 195.74509592874398}
{'trader_id': 428, 'price': 195.43623748035202}
{'trader_id': 91, 'price': 195.29318448500646}
{'trader_id': 705, 'price': 195.14747041390143}
{'trader_id': 33, 'price': 194.95036534548333}
{'t

{'trader_id': 498, 'price': 201.05152477570908}
{'trader_id': 1, 'price': 200.57609005351622}
{'trader_id': 461, 'price': 200.43932554425044}
{'trader_id': 168, 'price': 199.5543481843553}
{'trader_id': 329, 'price': 199.22567991789094}
{'trader_id': 713, 'price': 199.18259554886782}
{'trader_id': 555, 'price': 198.86710152954285}
{'trader_id': 790, 'price': 198.86638021393867}
{'trader_id': 563, 'price': 198.67941961636996}
{'trader_id': 438, 'price': 198.6165575260751}
{'trader_id': 123, 'price': 198.4521484164236}
{'trader_id': 126, 'price': 198.26086608648114}
{'trader_id': 466, 'price': 197.84539913370503}
{'trader_id': 457, 'price': 197.6757451077879}
{'trader_id': 677, 'price': 197.3076823923814}
{'trader_id': 6, 'price': 197.20666247606698}
{'trader_id': 737, 'price': 197.20338856705922}
{'trader_id': 741, 'price': 196.69389649334653}
{'trader_id': 682, 'price': 196.0167745770529}
{'trader_id': 330, 'price': 195.945442795387}
{'trader_id': 181, 'price': 195.74509592874398}
{'tr

{'trader_id': 62, 'price': 226.5475584383048}
{'trader_id': 90, 'price': 224.91054118237165}
{'trader_id': 478, 'price': 224.2585226868123}
{'trader_id': 610, 'price': 221.85706137196485}
{'trader_id': 621, 'price': 220.57623368519103}
{'trader_id': 700, 'price': 220.02779271076366}
{'trader_id': 15, 'price': 219.1682700965373}
{'trader_id': 247, 'price': 219.13536577764077}
{'trader_id': 513, 'price': 217.5924126050194}
{'trader_id': 143, 'price': 216.16496397993313}
{'trader_id': 248, 'price': 215.58660348567747}
{'trader_id': 192, 'price': 214.01064142732892}
{'trader_id': 60, 'price': 212.80679264103978}
{'trader_id': 40, 'price': 212.1777118686614}
{'trader_id': 258, 'price': 211.2772480043743}
{'trader_id': 479, 'price': 210.39597751847248}
{'trader_id': 370, 'price': 210.318381783142}
{'trader_id': 266, 'price': 209.66926211607225}
{'trader_id': 412, 'price': 209.45345786194036}
{'trader_id': 39, 'price': 208.60674384917075}
{'trader_id': 684, 'price': 208.4230229000023}
{'trade

{'trader_id': 478, 'price': 224.2585226868123}
{'trader_id': 610, 'price': 221.85706137196485}
{'trader_id': 621, 'price': 220.57623368519103}
{'trader_id': 700, 'price': 220.02779271076366}
{'trader_id': 15, 'price': 219.1682700965373}
{'trader_id': 247, 'price': 219.13536577764077}
{'trader_id': 513, 'price': 217.5924126050194}
{'trader_id': 143, 'price': 216.16496397993313}
{'trader_id': 248, 'price': 215.58660348567747}
{'trader_id': 192, 'price': 214.01064142732892}
{'trader_id': 60, 'price': 212.80679264103978}
{'trader_id': 40, 'price': 212.1777118686614}
{'trader_id': 258, 'price': 211.2772480043743}
{'trader_id': 479, 'price': 210.39597751847248}
{'trader_id': 370, 'price': 210.318381783142}
{'trader_id': 266, 'price': 209.66926211607225}
{'trader_id': 412, 'price': 209.45345786194036}
{'trader_id': 39, 'price': 208.60674384917075}
{'trader_id': 684, 'price': 208.4230229000023}
{'trader_id': 37, 'price': 208.3911270953931}
{'trader_id': 476, 'price': 207.8234011976399}
{'trade

{'trader_id': 613, 'price': 179.53276233270967}
{'trader_id': 144, 'price': 179.3389727734454}
{'trader_id': 533, 'price': 179.0278125833265}
{'trader_id': 471, 'price': 178.42212590496925}
{'trader_id': 231, 'price': 178.41559865116886}
{'trader_id': 446, 'price': 178.18030484591426}
{'trader_id': 328, 'price': 177.472045955779}
{'trader_id': 55, 'price': 177.09794458479882}
{'trader_id': 182, 'price': 177.00604347572516}
{'trader_id': 547, 'price': 176.68252989186823}
{'trader_id': 771, 'price': 176.5186059940514}
{'trader_id': 9, 'price': 176.2233951400222}
{'trader_id': 723, 'price': 176.1389500796099}
{'trader_id': 779, 'price': 175.59815218181095}
{'trader_id': 204, 'price': 175.51171299250564}
{'trader_id': 282, 'price': 175.18700956596462}
{'trader_id': 659, 'price': 174.81941043003795}
{'trader_id': 207, 'price': 174.72532734222415}
{'trader_id': 778, 'price': 174.7027767154919}
{'trader_id': 336, 'price': 174.65419056969867}
{'trader_id': 583, 'price': 174.51389149176362}
{'t

{'trader_id': 605, 'price': 191.39548561889575}
{'trader_id': 215, 'price': 191.1699835394133}
{'trader_id': 313, 'price': 191.0763872821758}
{'trader_id': 528, 'price': 190.97503833625765}
{'trader_id': 765, 'price': 190.9139210772584}
{'trader_id': 463, 'price': 190.50373464053965}
{'trader_id': 439, 'price': 190.35014542824393}
{'trader_id': 521, 'price': 190.3054809358992}
{'trader_id': 288, 'price': 190.10856055306556}
{'trader_id': 546, 'price': 190.05951555990328}
{'trader_id': 347, 'price': 190.03131848705607}
{'trader_id': 103, 'price': 190.02934363851008}
{'trader_id': 526, 'price': 189.85414767204463}
{'trader_id': 422, 'price': 189.6472919416338}
{'trader_id': 696, 'price': 189.54680990249912}
{'trader_id': 57, 'price': 189.37523694141785}
{'trader_id': 163, 'price': 189.182587961431}
{'trader_id': 553, 'price': 188.96431954056584}
{'trader_id': 776, 'price': 188.8884572019762}
{'trader_id': 369, 'price': 188.83494822698248}
{'trader_id': 394, 'price': 188.12310855272946}
{

In [26]:
past_price.pop()
past_price.append(current_market_price)
ob.agents_dict # Use this
for agent_id in range(800): 
    ep = ep_fundamentalist(current_market_price, P_F) 
    ob.agents_dict[i][4] = ep
    ob.agents_dict[i][5] = determine_order_type(ep,
                                                current_market_price, 
                                                TAO, ob.agents_dict[i][2],
                                                ob.agents_dict[i][3])

# Start from where N_FUND stops for integers
for i in range(800, 1600):
    ep = ep_chartist(current_market_price, past_price) 
    ob.agents_dict[i][4] = ep
    ob.agents_dict[i][5] = determine_order_type(ep,
                                                current_market_price, 
                                                TAO, ob.agents_dict[i][2],
                                                ob.agents_dict[i][3])

In [27]:
ob.agents_dict[805]

['chart', 40000, 35000, 50, 2827.0455195341765, 'buy', 0.7318371818108026]

In [ ]:
for agent_id in range(800, 1600):
    print(agents_dict[agent_id])